# Step 1: Install dependencies & log in to Hugging Face (token prompt)

In [ ]:
%pip install -q "datasets>=2.19.0" "huggingface_hub>=0.24"
from huggingface_hub import login
login(add_to_git_credential=True)

# Step 2: Load FLARE-FPB test set and normalize labels

In [ ]:
from datasets import load_dataset, Dataset

LABELS = ["negative","neutral","positive"]

ds_raw = load_dataset("TheFinAI/flare-fpb", split="test")
print("Loaded flare-fpb test:", len(ds_raw), "columns:", ds_raw.column_names)

_alias = {"pos":"positive","neg":"negative","neu":"neutral",
          "bullish":"positive","bearish":"negative"}

def _norm_label(v):
    if v is None: return None
    if isinstance(v, (int,float)) or (isinstance(v,str) and v.isdigit()):
        i = int(v); return LABELS[i] if 0 <= i < len(LABELS) else None
    s = str(v).strip().lower(); s = _alias.get(s, s)
    return s if s in LABELS else None

def _map_row(x):
    text = x.get("text") or x.get("sentence") or x.get("content") or x.get("input") or ""
    lab  = _norm_label(x.get("label", x.get("labels", x.get("answer"))))
    return {"text": text, "choices": LABELS, "answer": lab}

ds = Dataset.from_list([{**r, **_map_row(r)} for r in ds_raw])
bad = [i for i,r in enumerate(ds) if r["answer"] not in LABELS]
print("Samples with unusable label:", len(bad))
assert len(bad) == 0, "Found unparseable labels; please check the field mapping."


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/499 [00:00<?, ?B/s]

(…)-00000-of-00001-ab9a3b4799b09589.parquet:   0%|          | 0.00/608k [00:00<?, ?B/s]

(…)-00000-of-00001-303e4ba2afe838d4.parquet:   0%|          | 0.00/154k [00:00<?, ?B/s]

(…)-00000-of-00001-8bd1e21c671fb670.parquet:   0%|          | 0.00/188k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/776 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/970 [00:00<?, ? examples/s]

Loaded flare-fpb test: 970 columns: ['id', 'query', 'answer', 'text', 'choices', 'gold']
Samples with unusable label: 0


# Step 3: Install dependencies, configure OpenAI, and record experiment metadata

In [ ]:
%pip install -q "openai==1.40.2" "httpx==0.27.2" "httpcore==1.0.5" \
               "pandas>=2.2.2" "tqdm>=4.66.4" "requests>=2.31.0"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.34.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.
firebase-admin 6.9.0 requires httpx[http2]==0.28.1, but you have httpx 0.27.2 which is incompatible.


In [ ]:
import os, getpass, json, time, platform
from importlib.metadata import version, PackageNotFoundError

try:
    LABELS
except NameError:
    LABELS = ["negative", "neutral", "positive"]

MODEL    = "o3-mini"
BASE_URL = "https://api.openai.com/v1"

api_key = os.getenv("OPENAI_API_KEY") or os.getenv("API_KEY")
if not api_key:
    api_key = getpass.getpass("Paste your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = api_key

run_tag   = f"flare_fpb_{MODEL.replace('/','_')}"
save_dir  = "/content"
pred_path = f"{save_dir}/{run_tag}_predictions.csv"
meta_path = f"{save_dir}/{run_tag}_metadata.json"

def ver(pkg: str) -> str:
    try:
        return version(pkg)
    except PackageNotFoundError:
        return "not-installed"

meta = {
    "dataset": "TheFinAI/flare-fpb",
    "split": "test",
    "labels": list(LABELS),
    "model": MODEL,
    "openai_sdk": ver("openai"),
    "httpx": ver("httpx"),
    "httpcore": ver("httpcore"),
    "datasets_version": ver("datasets"),
    "pandas": ver("pandas"),
    "tqdm": ver("tqdm"),
    "time_utc": time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime()),
    "python": platform.python_version(),
    "base_url": BASE_URL,
    "note": "Responses API; strict text-JSON protocol; retries+checkpoint"
}

os.makedirs(save_dir, exist_ok=True)
with open(meta_path, "w") as f:
    json.dump(meta, f, indent=2)

print("Meta saved ->", meta_path)
print("MODEL:", MODEL, "| BASE_URL:", BASE_URL)
print("OPENAI_API_KEY is set:", bool(os.environ.get("OPENAI_API_KEY")))


Paste your OpenAI API key: ··········
Meta saved -> /content/flare_fpb_o3-mini_metadata.json
MODEL: o3-mini | BASE_URL: https://api.openai.com/v1
OPENAI_API_KEY is set: True


# Step 4: Inference & evaluation loop (o3-mini, strict text→JSON, retries + checkpoint)


In [ ]:
import requests, json, os, re, time
import pandas as pd
from tqdm import tqdm

def _strip_code_fences(s: str) -> str:
    s = s.strip()
    if s.startswith("```"):
        s = re.sub(r"^```[a-zA-Z0-9_-]*\s*", "", s)
        s = re.sub(r"\s*```$", "", s)
    return s.strip()

def _extract_output_text(data: dict) -> str | None:
    t = data.get("output_text")
    if isinstance(t, str) and t.strip():
        return t
    for o in data.get("output", []):
        for p in o.get("content", []):
            if p.get("type") == "output_text":
                tt = p.get("text")
                if isinstance(tt, str) and tt.strip():
                    return tt
    return None

def _make_user_text(sentence: str, choices=("",)):
    return (
        "Task: classify the sentence into exactly one of these labels: "
        f"{', '.join(choices)}.\n\n"
        f"Sentence: {sentence}\n\n"
        "Return ONLY a JSON object on a single line, exactly in this form:\n"
        "{\"label\":\"negative|neutral|positive\"}\n"
        "No code fences, no extra text, no explanation."
    )

def ask_o3_textjson_once(sentence, choices=("negative","neutral","positive"), max_tok=128):
    url = f"{BASE_URL.rstrip('/')}/responses"
    headers = {
        "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}",
        "Content-Type": "application/json",
    }
    user_text = _make_user_text(sentence, choices)
    payload = {
        "model": MODEL,
        "input": [{
            "role": "user",
            "content": [{"type": "input_text", "text": user_text}]
        }],
        "max_output_tokens": int(max_tok),
        "reasoning": {"effort": "low"},
    }
    r = requests.post(url, headers=headers, json=payload, timeout=120)
    if r.status_code != 200:
        raise RuntimeError(f"Responses API error {r.status_code}: {r.text[:500]}")
    data = r.json()
    if data.get("status") == "incomplete":
        reason = (data.get("incomplete_details") or {}).get("reason")
        raise RuntimeError(f"incomplete:{reason}")

    txt = _extract_output_text(data)
    if not isinstance(txt, str) or not txt.strip():
        raise RuntimeError(f"No output_text in response. Snippet: {json.dumps(data)[:400]}")

    txt = _strip_code_fences(txt)
    obj = json.loads(txt)
    lab = obj.get("label")
    if lab not in choices:
        raise RuntimeError(f"Invalid label {lab!r}; raw json: {obj}")
    return lab

def ask_o3_textjson(sentence, choices=("negative","neutral","positive")):
    for max_tok in (128, 256, 512):
        delay = 1.0
        for attempt in range(5):
            try:
                return ask_o3_textjson_once(sentence, choices, max_tok=max_tok)
            except RuntimeError as e:
                msg = str(e)

                if "Responses API error 5" in msg or "server_error" in msg:
                    time.sleep(delay); delay = min(delay*2, 30); continue
                if "Responses API error 429" in msg:
                    time.sleep(delay); delay = min(delay*2, 60); continue
                if "incomplete:max_output_tokens" in msg:
                    break
                raise
    raise RuntimeError("Exhausted retries and token budgets for this sample.")


run_tag   = f"flare_fpb_{MODEL.replace('/','_')}"
save_dir  = "/content"
pred_path = f"{save_dir}/{run_tag}_predictions.csv"
err_path  = f"{save_dir}/{run_tag}_errors.csv"

rows_done = []
done_idx  = set()
if os.path.exists(pred_path):
    old = pd.read_csv(pred_path)
    if "row_idx" in old.columns:
        rows_done = old.to_dict("records")
        done_idx  = set(old["row_idx"].tolist())
        print(f"[resume] loaded {len(done_idx)} completed rows.")

err_rows = []
buf = []
save_every = 50

total = len(ds)
for i in tqdm(range(total)):
    if i in done_idx:
        continue
    x = ds[i]
    text = x["text"]
    gold = x["answer"]

    try:
        pred = ask_o3_textjson(text, LABELS)
        raw  = json.dumps({"label": pred})
    except Exception as e:
        pred = "UNKNOWN"
        raw  = f"ERROR: {type(e).__name__}: {e}"
        err_rows.append({"row_idx": i, "id": x.get("id", i), "error": raw, "text": text})

    buf.append({
        "row_idx": i,
        "id": x.get("id", i),
        "text": text,
        "pred_raw": raw,
        "pred": pred,
        "label": gold
    })

    if len(buf) % save_every == 0:
        out = pd.DataFrame(rows_done + buf).sort_values("row_idx")
        out.to_csv(pred_path, index=False)
        if err_rows:
            pd.DataFrame(err_rows).to_csv(err_path, index=False)
        print(f"[checkpoint] saved {len(out)}/{total} -> {pred_path}")

out = pd.DataFrame(rows_done + buf).sort_values("row_idx")
out.to_csv(pred_path, index=False)
if err_rows:
    pd.DataFrame(err_rows).to_csv(err_path, index=False)
print(f"[done] saved -> {pred_path}")
if os.path.exists(err_path):
    print(f"[errors] logged -> {err_path}")


  5%|▌         | 50/970 [03:17<1:02:29,  4.08s/it]

[checkpoint] saved 50/970 -> /content/flare_fpb_o3-mini_predictions.csv


 10%|█         | 100/970 [06:18<43:43,  3.02s/it]

[checkpoint] saved 100/970 -> /content/flare_fpb_o3-mini_predictions.csv


 15%|█▌        | 150/970 [09:12<1:22:14,  6.02s/it]

...

[checkpoint] saved 900/970 -> /content/flare_fpb_o3-mini_predictions.csv


 98%|█████████▊| 950/970 [53:33<00:52,  2.61s/it]

[checkpoint] saved 950/970 -> /content/flare_fpb_o3-mini_predictions.csv


100%|██████████| 970/970 [54:55<00:00,  3.40s/it]

[done] saved -> /content/flare_fpb_o3-mini_predictions.csv


# Step5: Compute Macro-F1 and Accuracy

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score

df = pd.read_csv(pred_path).sort_values("row_idx").drop_duplicates("row_idx", keep="last")
ok = df[df["pred"] != "UNKNOWN"].copy()

try:
    LABELS
except NameError:
    LABELS = ["negative","neutral","positive"]

f1ma = f1_score(ok["label"], ok["pred"], labels=LABELS, average="macro", zero_division=0)
acc  = accuracy_score(ok["label"], ok["pred"])

print(f"F1: {f1ma:.4f}, Accuracy: {acc:.4f}")


F1: 0.7737, Accuracy: 0.7897
